In [1]:
%display latex
%display default

## Précision numérique

Par défaut, les nombres flottants et les calculs flottants sont fait avec une précision de 53 bits

In [2]:
a= 3.0
a.parent()

Real Field with 53 bits of precision

In [3]:
a.precision()

53

In [4]:
(1.0/3.0).parent()

Real Field with 53 bits of precision

In [5]:
print(1/3, (1/3).parent())
print( (1/3).n(), (1/3).n().parent())
print(RR(1/3), RR(1/3).parent())


1/3 Rational Field
0.333333333333333 Real Field with 53 bits of precision
0.333333333333333 Real Field with 53 bits of precision


Les nombres flottants sont définis comme instance de l'objet `RealField`. On peut spécifier la précision de cette objet.

In [6]:
R=RealField()
R10=RealField(10)
R200=RealField(200)

R.precision(), R10.precision(), R200.precision()

(53, 10, 200)

In [7]:
a   = R(3.513)
a10 = R10(3.513)
a200= R200(3.513)

a^100, a10^100, a200^100

(3.69670656868857e54,
 3.6e54,
 3.6967065686885807935368708162449489665828918404074416864066e54)

Calcul de $0.3-0.2-0.1$ avec différentes précisions

In [8]:
a,b,c=.3,.2,.1
a10,b10,c10=R10(.3),R10(.2),R10(.1)
a200,b200,c200=R200(.3),R200(.2),R200(.1)
a-b-c,a10-b10-c10,a200-b200-c200

(-2.77555756156289e-17,
 -0.00012,
 7.7787690973264271339300800672251553007378152109014589163764e-62)

**Attention:** 
- si on ne précise pas RealField, la précision est calculée automatiquement.
- si les calculs sont mixtes (opérandes avec différentes précisions), les calculs sont fait dans la plus petite précision

In [9]:
a=0.5555555555554033333333323232323233232111117
a.precision(), a.parent()

(143, Real Field with 143 bits of precision)

In [10]:
a10-b200-c200, a10-b10-c10

(-0.00012, -0.00012)

In [11]:
a= 0.1e-20
b= 0.1
c= 0.3e-10
a+b+c-b,b-b+a+c,a+c+b-b

(3.00000024822111e-11, 3.00000000010000e-11, 3.00000024822111e-11)

**A RETENIR**
> - `RealField(p)` est l'objet définissant les approximations flottantes à précision $p$
> - `numerical_approx(expr, p)` ou `n(expr,p)` ou `expr.n(p)` retourne l'approximation flottante de l'expression `expr` à précision `p`
> - par défaut `RealField(53)` est utilisé (déjà défini dans `RR`)

## Exemple de problème de précision

### calcul de racine carré

In [48]:
a=51
b=a^61
print("l'entier b est codée sur "+str( b.nbits()) + " bits")
r=sqrt(b)
r

l'entier b est codée sur 347 bits


1686961934066707040236155036109474174986501789839001*sqrt(51)

In [53]:
s   = R(sqrt(b))
s200= R200(sqrt(b))
print(floor(s),floor(s200))
floor(s)^2 <= b <(floor(s)+1)^2,\
floor(s200)^2 <= b <=(floor(s200)+1)^2

12047317913813611992216795507023424262776490557440000 12047317913813610588023392961569665056411324497659166


(False, True)

## résolution de système d'équations linéaires 


In [65]:
N=2
MS=MatrixSpace(QQ,N)
VS=VectorSpace(QQ,N)
A=MS.random_element(density=1,num_bound=10,den_bound=10)
b=VS.random_element(num_bound=100,den_bound=100)
V=var(list("x"+ str(i) for i in range(N)))
X=vector(V)
eq=list((A*X)[i]==b[i] for i in range(N))

### La fonction `solve`

elle prend une liste d'équations symboliques et une liste d'inconnues

In [15]:
print("variables:",V)
print("Equations:",eq)
#for i in range(N):
#    print(eq[i])

variables: (x0, x1)
Equations: [-4/3*x0 + 2/5*x1 == (-9/38), 8*x0 + 7/2*x1 == (-12/7)]


In [16]:
sol=solve(eq, V)
sol

[[x0 == (1143/62776), x1 == (-4170/7847)]]

Est-il possible de faire la même chose mais de faire le calcul numériquement ?

In [17]:
eqf=list((A.n()*X)[i]==b[i].n() for i in range(N))
print("variables:",V)
print("Equations:",eqf)
#for i in range(N):
#    print(eqf[i])

variables: (x0, x1)
Equations: [-1.33333333333333*x0 + 0.400000000000000*x1 == -0.236842105263158, 8.00000000000000*x0 + 3.50000000000000*x1 == -1.71428571428571]


In [18]:
solf=solve(eqf,V)
solf

[[x0 == (1143/62776), x1 == (-4170/7847)]]

Quoi, le résultat est le même !!!

En fait, `solve` fait appel à une méthode de résolution symbolique. 
Cela implique que les réels sont transformées en nombre rationnel lors de la résolution. 

### l'algèbre linéaire

In [19]:
show(eq)

[-4/3*x0 + 2/5*x1 == (-9/38), 8*x0 + 7/2*x1 == (-12/7)]

In [20]:
show(A, (V), b)

[-4/3  2/5]
[   8  7/2] (x0, x1) (-9/38, -12/7)

In [21]:
y=A.solve_right(b)
show(y)

(1143/62776, -4170/7847)

Pareil mais en numérique

In [22]:
Af=A.numerical_approx()
bf=b.n()
show(Af,V,bf)

[-1.33333333333333 0.400000000000000]
[ 8.00000000000000  3.50000000000000] (x0, x1) (-0.236842105263158, -1.71428571428571)

In [23]:
yf=Af.solve_right(bf)
show(yf)

(0.0182075952593348, -0.531413278960112)

In [24]:
# difference des solutions
y.n()-yf

(-2.08166817117217e-17, -1.11022302462516e-16)

Tout pareil mais en plus grand

In [25]:
N=80
MS=MatrixSpace(QQ,N)
VS=VectorSpace(QQ,N)
A=MS.random_element(density=1,num_bound=100,den_bound=100)
b=VS.random_element(num_bound=100,den_bound=100)


In [68]:
show(A,b)
%time y=A.solve_right(b)
A.solve_right(b)

[-10  -7]
[5/6 1/4] (-15/26, -45/2)

CPU times: user 215 µs, sys: 4 µs, total: 219 µs
Wall time: 223 µs


(-9837/208, 7035/104)

In [71]:
y[0]

-9837/208

In [72]:
print("num(y[0])=",y[0].numerator(),'\n')
print("den(y[0])=",y[0].denominator())

num(y[0])= -9837 

den(y[0])= 208


In [29]:
Af=A.n()
bf=b.n()
%time yf= Af.solve_right(bf)

CPU times: user 543 ms, sys: 17 µs, total: 543 ms
Wall time: 551 ms


In [30]:
print("yf[0]=",yf[0])
#erreur commise par la résolution numérique
y[0]-yf[0], (Af*yf)[0]- bf[0]

yf[0]= 0.863655548974229


(-2.86981549635357e-12, 5.83605386239583e-12)

In [31]:
N=20
A=matrix(N,N,lambda i,j : 1/(i+j+1))
b=vector([ZZ.random_element() for i in range(N)])
show(A)
show(b)

20 x 20 dense matrix over Rational Field (use the '.str()' method to see the entries)

(1, 180, -1, -1, -1, -2, 2, 6, 0, 3, 9, 0, 1, -1, 0, -29, 1, 1, 3, 0)

In [32]:
Af=A.n()
bf=b.n()
%time y = A.solve_right(b)
%time yf= Af.solve_right(bf)

CPU times: user 2.87 ms, sys: 54 µs, total: 2.92 ms
Wall time: 2.9 ms
CPU times: user 14 ms, sys: 0 ns, total: 14 ms
Wall time: 13.8 ms


In [33]:
### le résultat avec les flottants est complètement faux
print("yf[0]=",yf[0])
(Af*yf)[0], bf[0]

yf[0]= -1.58909996452975e11


(-7.00217600000000e6, 1.00000000000000)

In [34]:
### le résultat avec les rationnels est exact
print("y[0]=",y[0])
(A*y)[0],b[0],A*y==b

y[0]= 10852599905060620


(1, 1, True)

**A RETENIR**
- la fonction `solve` est une méthode de résolution symbolique
- les calculs avec les flottants peuvent être plus rapides mais aussi faux (surtout en algèbre linéaire)
- les calculs avec les rationnels entrainent un grossissement des données mais le résultat est toujours correct

## Limite du calcul symbolique

In [35]:
var('a b c')
e1=a*c+b*c
e2=(a+b)*c
e1.show()
e2.show()
(e2 == e1.collect(c)).show()
(e1 == e2.expand()).show()
bool(e1==e2)

a*c + b*c

(a + b)*c

(a + b)*c == (a + b)*c

a*c + b*c == a*c + b*c

True

In [36]:
e=(a-b)^2-a^2+2*a*b-b^2
e.show()
bool(e==0)

(a - b)^2 - a^2 + 2*a*b - b^2

True

In [37]:
var('x')
e= cos(x)^2+sin(x)^2
e, e.simplify_trig()

(cos(x)^2 + sin(x)^2, 1)

In [38]:
e=sqrt(x^2)
bool(x==e), bool(x==e.simplify_full()),bool(x==e.canonicalize_radical())

(False, False, True)

### Explosion combinatoire des symboles

In [39]:
var('x')
A=Matrix(2,2,[[cos(x),sin(x)],[-sin(x),cos(x)]])
show("A=",A,"  det(A)=",A.determinant())

'A=' [ cos(x)  sin(x)]
[-sin(x)  cos(x)] '  det(A)=' cos(x)^2 + sin(x)^2

In [40]:
N=100
d=(A^N).determinant()
print("d=",d)

#A^N,(A^(N)).expand()

d= 16*(16*(256*(16*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2)^2*(4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (64*(4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (16*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2)^2)^2)*(128*(64*(4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (16*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2)^2)*(16*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2)*(4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 + (256*(16*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2)^2*(4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*si

In [41]:
%time print(bool(d==1))
%time print(bool(d.simplify_trig()==1))

True
CPU times: user 8.44 s, sys: 52.4 ms, total: 8.49 s
Wall time: 8.27 s
True
CPU times: user 2.36 s, sys: 15.9 ms, total: 2.38 s
Wall time: 2.32 s


### Pas toujours de solutions

In [42]:
var('x')
(x^2+1).roots()

[(-I, 1), (I, 1)]

In [43]:
E= (x+1)^5 - (x+2)^4
F= (x+1)^5 + (x+2)^4
EF=E*F
print(f"{E=}\n{F=}\n{EF=}")
EF.roots(x)

E=(x + 1)^5 - (x + 2)^4
F=(x + 1)^5 + (x + 2)^4
EF=((x + 1)^5 + (x + 2)^4)*((x + 1)^5 - (x + 2)^4)


[(-1/6*(1/2)^(1/3)*(3*sqrt(23)*sqrt(3) + 97)^(1/3)*(I*sqrt(3) + 1) - 13/3*(1/2)^(2/3)*(-I*sqrt(3) + 1)/(3*sqrt(23)*sqrt(3) + 97)^(1/3) - 1/3,
  1),
 (-1/6*(1/2)^(1/3)*(3*sqrt(23)*sqrt(3) + 97)^(1/3)*(-I*sqrt(3) + 1) - 13/3*(1/2)^(2/3)*(I*sqrt(3) + 1)/(3*sqrt(23)*sqrt(3) + 97)^(1/3) - 1/3,
  1),
 (1/3*(1/2)^(1/3)*(3*sqrt(23)*sqrt(3) + 97)^(1/3) + 26/3*(1/2)^(2/3)/(3*sqrt(23)*sqrt(3) + 97)^(1/3) - 1/3,
  1),
 (-1/2*I*sqrt(3) - 3/2, 1),
 (1/2*I*sqrt(3) - 3/2, 1)]

In [44]:
E= (x+1)^10 - (x+2)^9
F= (x+1)^10 + (x+2)^9
EF=E*F
print(f"{E=}\n{F=}\n{EF=}")
EF.roots(x) ## aucunes solution trouvées alors que c'est un polynôme

E=(x + 1)^10 - (x + 2)^9
F=(x + 1)^10 + (x + 2)^9
EF=((x + 1)^10 + (x + 2)^9)*((x + 1)^10 - (x + 2)^9)


RuntimeError: no explicit roots found

Pour résoudre on passe du calcul symbolique au calcul algébrique.

Pour cela on se place dans la structure mathématique des polynomes qui sont des expressions symboliques ayant de bonne propriétés: en particulier une représentation canonique !!!

Un polynôme est une expression de la forme : $a_0+a_1X+\dots+a_nX^n$ avec $n$ une valeur connue et les coefficients $\{a_0,a_1,\dots,a_n\}$ qui sont tous du même type (Entier, Rationnel, Complexe, ...).

$\Rightarrow$ le vecteur $[a_0,a_1,\dots,a_n]$ est une représentation canonique de ce polynôme.

In [73]:
# Ensemble des polynomes à coefficient dans QQ
# on choisit la variable X comme symbole des polynômes
PR.<X>=PolynomialRing(QQ) #ou PR.<X>=QQ[]

print("type de x: ",x.parent())
print("type de X: ",X.parent())
bool(X==x)

type de x:  Symbolic Ring
type de X:  Univariate Polynomial Ring in X over Rational Field


False

In [74]:
print((2*X+4).parent()) 
print((2*x+4).parent()) 
print((2*X+ cos(3)).parent())
print((2*X+0.5).parent()) # attention coercion vers les polynomes à coefficient dans RR
RR

Univariate Polynomial Ring in X over Rational Field
Symbolic Ring
Symbolic Ring
Univariate Polynomial Ring in X over Real Field with 53 bits of precision


Real Field with 53 bits of precision

In [75]:
# on convertit les expressions symboliques vers les polynomes en substituant 
# la variable x par celle des polynômes en X et en construisant le polynôme associé
polyE=PR(E.subs(x=X)) 
polyF=PR(F.subs(x=X))
polyEF=polyE*polyF
show(f"une expression symbolique: {E=}")
show(f"un polynomes: {polyE=}")


'une expression symbolique: E=(x + 1)^10 - (x + 2)^9'

'un polynomes: polyE=X^10 + 9*X^9 + 27*X^8 - 24*X^7 - 462*X^6 - 1764*X^5 - 3822*X^4 - 5256*X^3 - 4563*X^2 - 2294*X - 511'

In [76]:
# calcul des racines dans le domaine de définition du polynome (ici Q)
print(f"calcul des racines sur {polyEF.base_ring()}")
polyEF.roots()

calcul des racines sur Rational Field


[]

In [77]:
# calcul des racines dans C
R=polyEF.roots(CC)
R

[(-1.51825014565097, 1),
 (4.06351076609262, 1),
 (-1.51929008466337 - 0.389595992999977*I, 1),
 (-1.51929008466337 + 0.389595992999977*I, 1),
 (-1.51922405950470 - 0.270825166290452*I, 1),
 (-1.51922405950470 + 0.270825166290452*I, 1),
 (-1.51876244356380 - 0.171773267885010*I, 1),
 (-1.51876244356380 + 0.171773267885010*I, 1),
 (-1.51838700372900 - 0.0834870580994881*I, 1),
 (-1.51838700372900 + 0.0834870580994881*I, 1),
 (-1.51739709419704 - 0.543838798817926*I, 1),
 (-1.51739709419704 + 0.543838798817926*I, 1),
 (-1.50785565319868 - 0.765883493953419*I, 1),
 (-1.50785565319868 + 0.765883493953419*I, 1),
 (-1.46510584891600 - 1.13480030613066*I, 1),
 (-1.46510584891600 + 1.13480030613066*I, 1),
 (-1.22672212879498 - 1.87991770546555*I, 1),
 (-1.22672212879498 + 1.87991770546555*I, 1),
 (0.520114006346739 - 3.18892454790237*I, 1),
 (0.520114006346739 + 3.18892454790237*I, 1)]

In [78]:
R[0][0],polyEF(R[0][0])

(-1.51825014565097, -9.48784872889519e-9)

In [79]:
polyEF(ComplexField(80)(R[0][0]))

6.3057198351756937881873e-16

In [80]:
# calcul de la forme canonique d'un produit de polynômes
E1= (x+1)^100 - (x+2)^99
F1= (x+1)^100 + (x+2)^99
polyE1=PR(E.subs(x=X)) 
polyF1=PR(F.subs(x=X))

# via la representation par arbre (exp symbolique)
%time (E1*F1).expand() 
# via la representation par liste des coefficients
%time polyEF1=polyE1*polyF1 

CPU times: user 35.5 ms, sys: 0 ns, total: 35.5 ms
Wall time: 35.8 ms
CPU times: user 0 ns, sys: 13 µs, total: 13 µs
Wall time: 16.9 µs


**A RETENIR**
- le calcul symbolique permet de manipuler n'importe quelles expressions faisant intervenir des symboles et des fonctions 
- pas toujours possible de trouver des solutions avec uniquement des calculs symboliques
- l'utilisation de calcul algèbrique est souvent préférable si on connaît la structure mathématique sous-jacente (Ex. les polynômes, les matrices, ...)
- le calcul algébrique sera la plupart du temps plus rapide
- parfois on retrouve un mélange de calcul algébrique et numérique (résolution dans les complexes si le problème est défini sur les nombres rationnels)

## Les matrices

In [81]:
A=matrix([[1,2],[3,4]])
B=matrix([[1,2,3],[4,5,6]])
print(A,B)
show(A,B)

[1 2]
[3 4] [1 2 3]
[4 5 6]


[1 2]
[3 4] [1 2 3]
[4 5 6]

In [82]:
# addition de matrice
show(A+A)
# multiplication par un scalaire
show(3*A , A+A+A)

[2 4]
[6 8]

[ 3  6]
[ 9 12] [ 3  6]
[ 9 12]

In [108]:
show("A=",A)
show("B=",B)

show(A+B)

'A=' [ 3 12  7  0 13  0 13 14  5  3]
[ 9  2  2  9 16  1  6  9  1 12]
[14 10 15  9  8  7 13  7  0  8]
[10  3 14 14  9  1 11  2  7  5]
[14 15 16  9  7 11 13  9  5 14]
[11  2 13  3  5  4 10 13 14 13]
[10  7  6  6  7  7 15  2 11  8]
[10  1  5  9 10  1  3  3 11  1]
[ 8 12  6  9 12  2  1 11  7 13]
[11  0  4 13 15 10  7 15  6  4]

'B=' [ 3 12  7  0 13  0 13 14  5  3]
[ 9  2  2  9 16  1  6  9  1 12]
[14 10 15  9  8  7 13  7  0  8]
[10  3 14 14  9  1 11  2  7  5]
[14 15 16  9  7 11 13  9  5 14]
[11  2 13  3  5  4 10 13 14 13]
[10  7  6  6  7  7 15  2 11  8]
[10  1  5  9 10  1  3  3 11  1]
[ 8 12  6  9 12  2  1 11  7 13]
[11  0  4 13 15 10  7 15  6  4]

[ 6  7 14  0  9  0  9 11 10  6]
[ 1  4  4  1 15  2 12  1  2  7]
[11  3 13  1 16 14  9 14  0 16]
[ 3  6 11 11  1  2  5  4 14 10]
[11 13 15  1 14  5  9  1 10 11]
[ 5  4  9  6 10  8  3  9 11  9]
[ 3 14 12 12 14 14 13  4  5 16]
[ 3  2 10  1  3  2  6  6  5  2]
[16  7 12  1  7  4  2  5 14  9]
[ 5  0  8  9 13  3 14 13 12  8]

### matrices et combinaisons linéaires

In [84]:
# une combinaison linéaire de symboles
var('x y z')
e=[x,y,z]
3*e[0]+2*e[1]+5*e[2]

3*x + 2*y + 5*z

In [85]:
u=matrix(1,3,[1,2,3])
v=matrix(3,1,e)
show(u,v)

[1 2 3] [x]
[y]
[z]

In [86]:
# le produit de matrice n*1 par 1*n est une combinaison linéaire
u*v

[x + 2*y + 3*z]

In [87]:
# on peut augmenter le nombre de combinaisons linéaires (on ajoute une ligne à gauche)
B=matrix(2,3,[1,2,3,4,5,6])
show(B,v)
show(B*v)

[1 2 3]
[4 5 6] [x]
[y]
[z]

[  x + 2*y + 3*z]
[4*x + 5*y + 6*z]

In [88]:
# on peut augmenter le nombre de combinaisons linéaires (on ajoute une colonne à droite)
var('x,y,z,a,b,c')
V=matrix(3,2,[[x,a],[y,b],[z,c]])
show(V)
show(B*V)

[x a]
[y b]
[z c]

[  x + 2*y + 3*z   a + 2*b + 3*c]
[4*x + 5*y + 6*z 4*a + 5*b + 6*c]

In [109]:
# ATTENTION: les tailles des matrices doivent correspondre
show(V)
V*V

[x a]
[y b]
[z c]

TypeError: unsupported operand parent(s) for *: 'Full MatrixSpace of 3 by 2 dense matrices over Symbolic Ring' and 'Full MatrixSpace of 3 by 2 dense matrices over Symbolic Ring'

In [110]:
# ATTENTION: le produit matriciel n'est pas commutatif
bool(V*B==B*V)

TypeError: unsupported operand parent(s) for *: 'Full MatrixSpace of 3 by 2 dense matrices over Symbolic Ring' and 'Full MatrixSpace of 10 by 10 dense matrices over Integer Ring'

In [111]:
show(V*B,B*V)

TypeError: unsupported operand parent(s) for *: 'Full MatrixSpace of 3 by 2 dense matrices over Symbolic Ring' and 'Full MatrixSpace of 10 by 10 dense matrices over Integer Ring'

In [112]:
# On peut transposer une matrice câd intervertir A[i,j] avec A[j,i]
show(V)
show(V.transpose())

[x a]
[y b]
[z c]

[x y z]
[a b c]

In [113]:
# le produit d'une matrice par sa transposée est toujours bien défini
show(V.transpose()*V)
show(V*V.transpose())

[x^2 + y^2 + z^2 a*x + b*y + c*z]
[a*x + b*y + c*z a^2 + b^2 + c^2]

[a^2 + x^2 a*b + x*y a*c + x*z]
[a*b + x*y b^2 + y^2 b*c + y*z]
[a*c + x*z b*c + y*z c^2 + z^2]

### Les espaces de matrices

In [114]:
# le premier paramètres est le domaine des coefficients (optionnel)
# si il est présent il impose le type des coefficients
A=matrix([[1,2],[3,4]])
B=matrix(RR,[[1,2/3],[3,4]])
C=matrix([[1,2/3],[3,4]])
A.parent(),B.parent(),C.parent()

(Full MatrixSpace of 2 by 2 dense matrices over Integer Ring,
 Full MatrixSpace of 2 by 2 dense matrices over Real Field with 53 bits of precision,
 Full MatrixSpace of 2 by 2 dense matrices over Rational Field)

In [115]:
matrix(ZZ,[[1,2/3],[3,4]])

TypeError: no conversion of this rational to integer

In [118]:
# le second/troisièmes paramètres sont les dimensions (optionnel)
# si ils sont présents ils imposent le nombre de coefficients (si aucun que des zéros)
A=matrix(ZZ,2,2)
B=matrix(ZZ,2,2,[[4,5],[6,7]])
C=matrix(ZZ,2,2,[4,5,6,7])
show(A)
show(B)
show(C)

[0 0]
[0 0]

[4 5]
[6 7]

[4 5]
[6 7]

In [119]:
# Comme les matrices vivent dans un espace de matrice on peut définir cet objet
MS23=MatrixSpace(ZZ,2,3)
MS22=MatrixSpace(QQ,2,2)
show(MS22,"\tet\t",MS23)
MS22,MS23

Full MatrixSpace of 2 by 2 dense matrices over Rational Field '\tet\t' Full MatrixSpace of 2 by 3 dense matrices over Integer Ring

(Full MatrixSpace of 2 by 2 dense matrices over Rational Field,
 Full MatrixSpace of 2 by 3 dense matrices over Integer Ring)

In [120]:
MS23.dimension() , MS23.dims()

(6, (2, 3))

In [121]:
A=MS22.matrix([1,2,3,4])
A.parent()

Full MatrixSpace of 2 by 2 dense matrices over Rational Field

In [122]:
MS22.random_element()
show(_)

Full MatrixSpace of 2 by 2 dense matrices over Rational Field

In [123]:
MS10=MatrixSpace(ZZ,10,10)
MS10.random_element(density=1,x=0,y=100)
show(_)

Full MatrixSpace of 2 by 2 dense matrices over Rational Field

In [124]:
MS10=MS10.change_ring(QQ)
MS10.random_element(density=0.5,num_bound=100,den_bound=100)
show(_)

Full MatrixSpace of 2 by 2 dense matrices over Rational Field

### un peu de calcul algébrique matriciel

In [125]:
N=10
q=17
MS=MatrixSpace(GF(q),N,N)
A=MS.random_element()
show("A=",A)
show(f"det(A)={A.det()}")

'A=' [13  1  1 13  5  1  9  9 13  1]
[12 10 12  4  9 13  1  6  9 15]
[ 9  4  8 15  0 11 13 12  5  1]
[ 5 10  8  4 14  6  1 14  3 14]
[ 1 13  9 13  7  2  0 12 16  5]
[11  2 16  6  9 12  0  1  3  7]
[16 15  9 14  0  6  1  1  6 14]
[ 7  9  9  6 16  8  4  4  1 12]
[ 7  4  5  9 12  4 11  8 13 15]
[11 14  9  1  8 12  9 11 10  6]

'det(A)=15'

In [126]:
B=A.change_ring(ZZ)
show("B=",B)
show(f"det(B)={B.det()}")

'B=' [13  1  1 13  5  1  9  9 13  1]
[12 10 12  4  9 13  1  6  9 15]
[ 9  4  8 15  0 11 13 12  5  1]
[ 5 10  8  4 14  6  1 14  3 14]
[ 1 13  9 13  7  2  0 12 16  5]
[11  2 16  6  9 12  0  1  3  7]
[16 15  9 14  0  6  1  1  6 14]
[ 7  9  9  6 16  8  4  4  1 12]
[ 7  4  5  9 12  4 11  8 13 15]
[11 14  9  1  8 12  9 11 10  6]

'det(B)=163526104283'

In [127]:
B.det()%q == A.det()

True

In [128]:
show(A^-1)

[13  0  3  4 16  7 14 15  2 15]
[12  6 13 12 15  9 13  1  9 11]
[ 5 15  8 15 16 13 12 15 11  4]
[ 8 10 14  4  9  3 15  9 10  9]
[14  4 10  0 16 12 15  6 16  2]
[ 6  2  5  4 14  6  7 11 12 13]
[16  0 14  1 11  2  2 13 15 15]
[ 2  7  2  6  6  5 13 15 11  8]
[12 12 13  1 10  0 11  7 12 13]
[14 16  2 10 15  8  3  2  5  5]

In [129]:
show(B^-1)

[  9374237486/163526104283  -4805561445/163526104283  -5551077037/163526104283   5659144051/163526104283  -6664768585/163526104283   5166878224/163526104283   4981380143/163526104283  -4716123812/163526104283  -3588020973/163526104283   4477069701/163526104283]
[ -2752556520/163526104283  -4141274302/163526104283  -1574504462/163526104283  -3102225491/163526104283   3285736969/163526104283  -5417961711/163526104283   4593501668/163526104283   6927908032/163526104283  -3850803740/163526104283   6948752647/163526104283]
[-11107925215/163526104283 -31572015779/163526104283  -5194614604/163526104283   9663230325/163526104283   1285250696/163526104283  26441181344/163526104283   8284902172/163526104283 -18709630615/163526104283  13169671872/163526104283  12344147993/163526104283]
[  3211650524/163526104283   8355268933/163526104283   6906874705/163526104283  -5472656328/163526104283   4228979312/163526104283  -6962085741/163526104283   -925413866/163526104283  11053227160/163526104283  -5349657821/163526104283 -11779788581/163526104283]
[  6494643966/163526104283   2644949667/163526104283  -3284206169/163526104283  -3299549363/163526104283   1561419181/163526104283  -2360371943/163526104283  -5941604121/163526104283  14634984774/163526104283  -4601601274/163526104283  -1898137418/163526104283]
[  2737077695/163526104283  36794991705/163526104283  11513045259/163526104283 -14304407061/163526104283   1382373264/163526104283 -18804216914/163526104283 -12398553438/163526104283  16721876901/163526104283 -13142157129/163526104283 -11857677557/163526104283]
[ -7175174341/163526104283 -18540162685/163526104283   1280012626/163526104283  -1150292167/163526104283  -3557196285/163526104283   5700717827/163526104283   3970953360/163526104283  -3087029924/163526104283  13010466184/163526104283  10712778042/163526104283]
[  2035870118/163526104283  -2312855250/163526104283   2437707448/163526104283  14441968627/163526104283  -1516549510/163526104283   1045335260/163526104283  -1195655059/163526104283 -10578850385/163526104283  -2034366562/163526104283    416327502/163526104283]
[  2342972429/163526104283   9516350314/163526104283  -3076150757/163526104283  -7883442455/163526104283   5151150226/163526104283  -2007341555/163526104283  -3997774201/163526104283  -2406695794/163526104283   2726657533/163526104283    100180515/163526104283]
[ -5645159207/163526104283   3511960030/163526104283    582214143/163526104283   4676472418/163526104283  -3396437068/163526104283  -1698482851/163526104283   3493988506/163526104283  -6217453609/163526104283   8789675162/163526104283  -4473487144/163526104283]

In [130]:
(B^(-1)%q), A^-1
show(_)

True

In [131]:
# Expliquer le résultat obtenu
(B^(-1)%q) ==  A^-1

False

In [132]:
(B^(-1)%q).parent(),(A^-1).parent()

(Full MatrixSpace of 10 by 10 dense matrices over Rational Field,
 Full MatrixSpace of 10 by 10 dense matrices over Finite Field of size 17)

In [133]:
(B^(-1)%q) ==  (A^-1).change_ring(B.base_ring())

True

In [134]:
(B^(-1)).change_ring(A.base_ring()) ==  (A^-1)

True